# FB-Prophet

In [39]:
# FB-Prophet
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import os

In [40]:
# Load dataset
df = pd.read_csv("data/interpolated_co2.csv")

In [41]:
# Strip whitespace and rename properly
df.rename(columns=lambda x: x.strip(), inplace=True)
df.rename(columns={"Date": "ds", "CO2": "y"}, inplace=True)

In [42]:
# Ensure proper types
df['ds'] = pd.to_datetime(df['ds'], errors='coerce')
df['y'] = pd.to_numeric(df['y'], errors='coerce')

In [43]:
# Drop missing values
df = df[['ds', 'y']].dropna()

In [44]:
# Split data into training and testing sets
train = df[df['ds'] <= '2024-12-31']
test = df[(df['ds'] >= '2025-01-01') & (df['ds'] <= '2025-04-13')]

In [45]:
# Training Prophet model with hyperparameter tuning
model = Prophet(
    changepoint_prior_scale=0.05,  # Allows the model to better fit the trend changes
    seasonality_prior_scale=10.0,  # Controls seasonality flexibility
    yearly_seasonality=True,       # CO2 emissions tend to have yearly seasonality
    weekly_seasonality=False,      # If weekly seasonality isn't needed
    daily_seasonality=False,       # If daily seasonality isn't needed
)

In [46]:
# Fit the model on the training data
model.fit(train)

15:29:33 - cmdstanpy - INFO - Chain [1] start processing
15:29:36 - cmdstanpy - INFO - Chain [1] done processing


In [47]:
# Make future dataframe and predictions
future = model.make_future_dataframe(periods=len(test), freq='D')
forecast = model.predict(future)

In [48]:
# Filter predictions to test period
forecast_test = forecast[forecast['ds'].isin(test['ds'])]

In [49]:
# Metrics
predicted = forecast_test['yhat'].values
actual = test['y'].values

In [50]:
# RMSE: Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(actual, predicted))

# MAE: Mean Absolute Error
mae = mean_absolute_error(actual, predicted)

# Accuracy: Percentage of correct predictions
accuracy = 100 - np.mean(np.abs((actual - predicted) / actual)) * 100

In [51]:
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"Accuracy: {accuracy:.2f}%")

RMSE: 1.2008223951333519
MAE: 0.9790045750232964
Accuracy: 99.77%


In [52]:
# Save predictions
predictions_df = pd.DataFrame({
    'Date': test['ds'],  # Use 'ds' for the Date column from test data
    'Actual': test['y'].values,  # Actual CO2 values from the test set
    'Predicted': forecast_test['yhat'].values  # Predicted CO2 values from the forecast
})

# Set 'Date' as the index
predictions_df.set_index('Date', inplace=True)

# Ensure the "Predictions" folder exists
os.makedirs("Predictions", exist_ok=True)

# Save the predictions to a CSV file
predictions_df.to_csv("Predictions/fb_predictions.csv")

print("Predictions saved to Predictions/fb_predictions.csv")


Predictions saved to Predictions/fb_predictions.csv
